In [31]:
import os

In [32]:
%pwd

'c:\\Users\\prass\\OneDrive\\Desktop\\practise\\cancer_project\\Chest-Cancer-Classification-using-Mlflow-DVC'

In [33]:
os.chdir("c:\\Users\\prass\\OneDrive\\Desktop\\practise\\cancer_project\\Chest-Cancer-Classification-using-Mlflow-DVC")

In [34]:
%pwd

'c:\\Users\\prass\\OneDrive\\Desktop\\practise\\cancer_project\\Chest-Cancer-Classification-using-Mlflow-DVC'

In [35]:
from dataclasses import dataclass
from pathlib import Path

In [36]:

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [26]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/p7-source/Chest-Cancer-Classification-using-Mlflow-DVC.mlflow"

In [37]:
import tensorflow as tf

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [38]:
%pwd

'c:\\Users\\prass\\OneDrive\\Desktop\\practise\\cancer_project\\Chest-Cancer-Classification-using-Mlflow-DVC'

In [39]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [46]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/MLdata",
            mlflow_uri="https://dagshub.com/p7-source/Chest-Cancer-Classification-using-Mlflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [59]:
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [60]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

In [61]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [63]:
import dagshub
dagshub.init(repo_owner='p7-source', repo_name='Chest-Cancer-Classification-using-Mlflow-DVC', mlflow=True)
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    import dagshub
    dagshub.init(repo_owner='p7-source', repo_name='Chest-Cancer-Classification-using-Mlflow-DVC', mlflow=True)
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-26 14:14:58,342: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/p7-source/Chest-Cancer-Classification-using-Mlflow-DVC "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "p7-source/Chest-Cancer-Classification-using-Mlflow-DVC"

[2024-12-26 14:14:58,414: INFO: helpers: Initialized MLflow to track repo "p7-source/Chest-Cancer-Classification-using-Mlflow-DVC"]


Repository p7-source/Chest-Cancer-Classification-using-Mlflow-DVC initialized!

[2024-12-26 14:14:58,417: INFO: helpers: Repository p7-source/Chest-Cancer-Classification-using-Mlflow-DVC initialized!]
[2024-12-26 14:14:58,444: INFO: common: yaml file: c:\Users\prass\OneDrive\Desktop\practise\cancer_project\Chest-Cancer-Classification-using-Mlflow-DVC\config\config.yaml loaded successfully]
[2024-12-26 14:14:58,463: INFO: common: yaml file: c:\Users\prass\OneDrive\Desktop\practise\cancer_project\Chest-Cancer-Classification-using-Mlflow-DVC\params.yaml loaded successfully]
[2024-12-26 14:14:58,470: INFO: common: created directory at: artifacts]
Found 52 images belonging to 2 classes.
4/4 [==============================] - 7s 1s/step - loss: 0.1081 - accuracy: 0.9423
[2024-12-26 14:15:06,843: INFO: common: json file saved at: scores.json]
[2024-12-26 14:15:07,612: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/p7-source/Chest-Cancer-Classification-using-Mlflow-DVC "HTTP/1.1 200 OK"]


...

Repository p7-source/Chest-Cancer-Classification-using-Mlflow-DVC initialized!

[2024-12-26 14:15:07,630: INFO: helpers: Repository p7-source/Chest-Cancer-Classification-using-Mlflow-DVC initialized!]


2024/12/26 14:15:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-26 14:15:09,799: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-12-26 14:15:10,561: INFO: builder_impl: Assets written to: C:\Users\prass\AppData\Local\Temp\tmpp2xwmegf\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/12/26 14:16:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
